# Sarter Notebook

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import haversine as hs

## Reading Data

In [2]:
# read train data
df_orders = pd.read_csv("Datasets/orders.csv", sep=";")
df_products = pd.read_csv("Datasets/product_attributes.csv", sep=",")
df_dists = pd.read_csv("Datasets/cities_data.csv", sep=";")
df_ordersXproducts = pd.read_csv('Datasets/ordersXproducts.csv')
del df_ordersXproducts['Unnamed: 0']

# read test data
df_test = pd.read_csv('Datasets/finaltest.csv')
df_test['weight'] = df_test['weight'].fillna(df_test['weight'].mean())
df_test['late_order'] = True

## Merge tables

In [3]:
df_coord = df_dists[['city_from_name', 'city_from_coord']].drop_duplicates().reset_index().drop(columns=['index']).copy()

In [4]:
df_alt = df_dists.copy()

c2 = df_alt['city_from_name'].values.copy()
c1 = df_alt['city_to_name'].values.copy()
c4 = df_alt['city_from_coord'].values.copy()
c3 = df_alt['city_to_coord'].values.copy()

df_alt['city_from_name'] = c1
df_alt['city_to_name'] = c2
df_alt['city_from_coord'] = c3
df_alt['city_to_coord'] = c4

df_dists = pd.concat([df_dists, df_alt]).reset_index().drop(columns=['index'])

In [5]:
df = pd.merge(df_ordersXproducts, df_dists, how='left', left_on=['origin_port', 'logistic_hub'], right_on=['city_from_name', 'city_to_name'])
del df['city_from_name']
del df['city_to_name']
del df['city_from_coord']
del df['city_to_coord']
df = df.rename(columns={'distance': 'dist_origin_hub'})

df = pd.merge(df, df_dists, how='left', left_on=['logistic_hub', 'customer'], right_on=['city_from_name', 'city_to_name'])
del df['city_from_name']
del df['city_to_name']
del df['city_from_coord']
del df['city_to_coord']
df = df.rename(columns={'distance': 'dist_hub_customer'})

df = pd.merge(df, df_dists, how='left', left_on=['origin_port', 'customer'], right_on=['city_from_name', 'city_to_name'])
del df['city_from_name']
del df['city_to_name']
del df['city_from_coord']
del df['city_to_coord']
df = df.rename(columns={'distance': 'dist_origin_customer'})

df = pd.merge(df, df_coord, how='left', left_on=['origin_port'], right_on=['city_from_name'])
del df['city_from_name']
df = df.rename(columns={'city_from_coord': 'origin_port_coord'})

df = pd.merge(df, df_coord, how='left', left_on=['logistic_hub'], right_on=['city_from_name'])
del df['city_from_name']
df = df.rename(columns={'city_from_coord': 'logistic_hub_coord'})

df = pd.merge(df, df_coord, how='left', left_on=['customer'], right_on=['city_from_name'])
del df['city_from_name']
df = df.rename(columns={'city_from_coord': 'customer_coord'})

In [6]:
for i in range(len(df)):
    if (not pd.isna(df.at[i, 'dist_origin_hub'])) and (not pd.isna(df.at[i, 'dist_hub_customer'])):
        df.at[i, 'dist_origin_customer'] = df.at[i, 'dist_origin_hub'] + df.at[i, 'dist_hub_customer']

In [7]:
x = df[~df['logistic_hub'].isna()]
indexes = x[x['dist_hub_customer'].isna()].index

for i in indexes:
    df.at[i, 'dist_hub_customer'] = 0

In [8]:
x = df[df['dist_origin_customer'].isna()]
indexes = x.index

for i in indexes:
    df.at[i, 'dist_origin_customer'] = 0

In [9]:
x = df[~df['logistic_hub'].isna()]
indexes = x[x['dist_hub_customer'].isna()].index

for i in indexes:
    df.at[i, 'dist_hub_customer'] = 0

In [10]:
x = df[df['dist_origin_customer'].isna()]
indexes = x.index

for i in indexes:
    df.at[i, 'dist_origin_customer'] = 0

In [11]:
df.to_csv('Datasets/finaldf.csv', index=False)

# Data preprocessing

In [46]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
import torch
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate

In [53]:
df = pd.read_csv('Datasets/finaldf.csv')
not_useful = ['order_id', 'origin_port_coord',
       'logistic_hub_coord', 'customer_coord', 'logistic_hub', 'material_handling']
for col in not_useful:
    del df[col]
df['weight'] = df['weight'].fillna(df['weight'].mean())
df = df.dropna().reset_index().drop(columns=['index'])

In [39]:
"""
units, weight = df['units'].values, df['weight'].values
df['total_weight']= units*weight
"""

In [54]:
categorical_columns = ['origin_port', '3pl', 'customs_procedures', 'product_id', 'customer']
numerical_columns = ['units', 'weight', 'dist_origin_customer']

target = df['late_order']
data = df.drop(columns=['late_order'])

In [55]:
categorical_preprocessor = OneHotEncoder(handle_unknown="ignore")
numerical_preprocessor = StandardScaler()

preprocessor = ColumnTransformer([
    ('one-hot-encoder', categorical_preprocessor, categorical_columns),
    ('standard_scaler', numerical_preprocessor, numerical_columns)])



In [56]:
model = Pipeline(steps=[('preprocessor', preprocessor), 
                        ('classifier', LogisticRegression(max_iter=50))])
cv_results = cross_validate(model, data, target, cv=5)
cv_results['test_score'].mean()

/home/jonathan/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/jonathan/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regr

0.8173580599161884

In [57]:
x = np.squeeze(np.asarray(preprocessor.fit_transform(data).todense()))
y = list(target.values)
y = [[e] for e in y]
y = torch.tensor(y, dtype=torch.float32)

In [58]:
padding = np.zeros(1024-len(x[0]))

In [59]:
z = []
for i in range(len(x)):
    z.append(list(np.concatenate((x[i], padding))))
x = torch.tensor(z, dtype=torch.float32)

In [60]:
model.fit(data, target)

/home/jonathan/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('one-hot-encoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['origin_port', '3pl',
                                                   'customs_procedures',
                                                   'product_id', 'customer']),
                                                 ('standard_scaler',
                                                  StandardScaler(),
                                                  ['units', 'weight',
                                                   'dist_origin_customer'])])),
                ('classifier', LogisticRegression(max_iter=50))])

In [61]:
model['classifier'].coef_

array([[ 2.32415549e-01, -1.29693332e-01, -6.08636427e-01,
         6.08632474e-01, -6.58391621e-01, -2.23056801e-01,
        -2.33098262e-01, -5.53974296e-01, -1.59687327e-01,
         2.07747413e-01,  1.08783873e-01,  1.63058339e+00,
        -7.94212962e-01, -5.83215901e-02, -4.14121251e-01,
        -3.87875220e-01, -5.12304907e-02, -1.54013153e-01,
        -4.93616710e-01, -6.25865384e-01, -8.68879661e-02,
        -6.28197735e-01, -6.08472820e-01,  9.43374659e-02,
        -2.72852712e-01, -3.82394992e-02,  5.35013571e-01,
        -3.32640897e-01,  3.07378131e-01, -3.18222263e-01,
        -2.60751053e-01, -7.80936792e-01, -2.17411438e-01,
         2.26349989e+00, -8.25119807e-01, -2.79037193e-01,
        -3.44961444e-01, -4.84245178e-01, -1.93218173e-01,
        -4.40732341e-01, -3.32070504e-01, -9.93665528e-01,
        -2.59940849e-01,  2.67739706e-01,  1.47224139e+00,
        -8.98983961e-01, -1.95589054e-01, -6.84485959e-01,
         1.66729285e+00,  2.02233161e+00, -2.38955081e-0

In [62]:
LogisticRegressionProbs = model.predict_proba(df_test.drop(columns=not_useful+['late_order']))[:, 1]

In [63]:
LogisticRegressionProbs

array([0.1243    , 0.14126935, 0.23928753, ..., 0.12044055, 0.38198951,
       0.21665572])

In [ ]:
df_pred_LR = pd.DataFrame()
df_pred_LR['order_id'] = df_test['order_id']
df_pred_LR['late_order'] = LogisticRegressionProbs

In [ ]:
df_pred_LR.to_csv('Predictions/pred_LogisticRegression4.csv', index=False)

# Papi DL

In [ ]:
import math

import torch
import torch.nn as nn

from sklearn.model_selection import train_test_split

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [ ]:
d_model = len(x[0])

class Net(nn.Module):
    """
    Text classifier based on a pytorch TransformerEncoder.
    """

    def __init__(
        self,
        nhead=4,
        dim_feedforward=1024,
        num_layers=4,
        dropout=0.1,
        activation="relu",
        classifier_dropout=0.1,
    ):

        super().__init__()

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(
            encoder_layer,
            num_layers=num_layers,
        )
        self.classifier = nn.Linear(d_model, 1)
        self.d_model = d_model

    def forward(self, x):
        x = self.transformer_encoder(x)
        x = torch.sigmoid(self.classifier(x))

        return x

In [ ]:
batch = 0
epoch_loss = 0
epoch_correct = 0
epoch_count = 0
predictions = model(x_train[batch*batch_size : (batch+1)*batch_size].to(device))
labels = y_train[batch*batch_size : (batch+1)*batch_size].to(device)

loss = criterion(predictions, labels)

correct = predictions.round() == labels
acc = correct.sum().item() / correct.size(0)

epoch_correct += correct.sum().item()
epoch_count += correct.size(0)

NameError: name 'x_train' is not defined

In [ ]:
epoch_correct / epoch_count

0.9052734375

In [ ]:
epochs = 4
model = Net().to(device)

criterion = nn.BCELoss()

lr = 1e-4
optimizer = torch.optim.Adam(
    (p for p in model.parameters() if p.requires_grad), lr=lr
)

torch.manual_seed(0)

batch_size=1024
batches = math.ceil(len(x)/batch_size)

print("starting")
for epoch in range(epochs):
    print(f"{epoch=}")
    epoch_loss = 0
    epoch_correct = 0
    epoch_count = 0
    for batch in range(math.ceil(len(x)/batch_size)):
        optimizer.zero_grad()
        predictions = model(x[batch*batch_size : (batch+1)*batch_size].to(device))
        labels = y[batch*batch_size : (batch+1)*batch_size].to(device)

        loss = criterion(predictions, labels)

        correct = predictions.round() == labels
        acc = correct.sum().item() / correct.size(0)

        epoch_correct += correct.sum().item()
        epoch_count += correct.size(0)

        epoch_loss += loss.item()

        loss.backward()

        optimizer.step()
    """
        predictions = model(x_train[batch*batch_size : (batch+1)*batch_size].to(device))
        labels = y_train[batch*batch_size : (batch+1)*batch_size].to(device)

        loss = criterion(predictions, labels)

        correct = predictions.round() == labels
        acc = correct.sum().item() / correct.size(0)

        epoch_correct += correct.sum().item()
        epoch_count += correct.size(0)

        epoch_loss += loss.item()

        loss.backward()

        optimizer.step()
    
    with torch.no_grad():
        test_epoch_loss = 0
        test_epoch_correct = 0
        test_epoch_count = 0

        for batch in range(math.ceil(len(x_test)/batch_size)):
            predictions = model(x_test[batch*batch_size : (batch+1)*batch_size].to(device))
            labels = y_test[batch*batch_size : (batch+1)*batch_size].to(device)
            test_loss = criterion(predictions, labels)

            correct = predictions.round() == labels
            acc = correct.sum().item() / correct.size(0)

            test_epoch_correct += correct.sum().item()
            test_epoch_count += correct.size(0)
            test_epoch_loss += loss.item()

    print(f"{epoch_loss=}")
    print(f"epoch accuracy: {epoch_correct / epoch_count}")
    print(f"{test_epoch_loss=}")
    print(f"test epoch accuracy: {test_epoch_correct / test_epoch_count}")
    """
    print(f"{epoch_loss=}")
    print(f"epoch accuracy: {epoch_correct / epoch_count}")

starting
epoch=0
epoch_loss=51.87281847000122
epoch accuracy: 0.7938537163267291
epoch=1
epoch_loss=45.233750373125076
epoch accuracy: 0.8235153701296979
epoch=2
epoch_loss=44.430546551942825
epoch accuracy: 0.8265036380836185
epoch=3
epoch_loss=43.90933087468147
epoch accuracy: 0.828775782652132


In [ ]:
torch.save(model.state_dict(), 'DLmodel/PapiDL.model')

In [ ]:
data = df_test.drop(columns=not_useful+['late_order'])

k = np.squeeze(np.asarray(preprocessor.fit_transform(data).todense()))

padding = np.zeros(1024-len(k[0]))

z = []
for i in range(len(k)):
    z.append(list(np.concatenate((k[i], padding))))
k = torch.tensor(z, dtype=torch.float32)

In [ ]:
preds = []
batch_size=128
for batch in range(math.ceil(len(k)/batch_size)):
    preds.append(model(k[batch*batch_size : (batch+1)*batch_size].to(device)))

In [ ]:
preds = torch.cat(list(torch.cat(preds))).detach().numpy()

In [ ]:
def trans(num):
    return 1 - num
t = np.vectorize(trans)
p = t(p)

In [ ]:
p

array([0.25205749, 0.30069175, 0.09594342, ..., 0.16871293, 0.19298422,
       0.08183663])

In [ ]:
df_pred_DL = pd.DataFrame()
df_pred_DL['order_id'] = df_test['order_id']
df_pred_DL['late_order'] = p

In [ ]:
df_pred_DL.to_csv('Predictions/pred_DL.csv', index=False)